In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

X=pd.read_csv("data limpia/X_definitiva.csv")
Y=pd.read_csv("data limpia/Y_definitiva.csv")
Y=Y["Precio"]
Z_test=pd.read_csv("data limpia/Z_test_definitiva.csv")

data=X.copy()
data["Price"]=Y

data.drop(columns={"Inches","screeninfo_1_20","memory_1_36","procesador_modelo_1_13","gpu_marca_1_3","gpu_modelo_1_10"},inplace=True)
Z_test.drop(columns={"Inches","screeninfo_1_20","memory_1_36","procesador_modelo_1_13","gpu_marca_1_3","gpu_modelo_1_10"},inplace=True)

X=data.copy()
X.drop(columns={"Price"},inplace=True)
Y=data["Price"]

In [2]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

In [3]:
from sklearn.metrics import mean_absolute_error
import xgboost

xgb_reg = xgboost.XGBRegressor(random_state=42)

xgb_reg.fit(X_train, Y_train)
y_pred = xgb_reg.predict(X_test)
mean_absolute_error(Y_test, y_pred) 

228.09407437350583

In [4]:
from sklearn.model_selection import GridSearchCV

In [5]:
depths=np.arange(0,8,1)
depths

array([0, 1, 2, 3, 4, 5, 6, 7])

In [6]:
estimators=np.arange(0,200,50)
estimators

array([  0,  50, 100, 150])

In [7]:
params = {"max_depth":depths,"n_estimators":estimators,"booster":["gbtree","gblinear"]}
bag_clf_gridsearch = GridSearchCV(xgb_reg, param_grid= params, scoring= "neg_mean_squared_error", cv = 5)

In [8]:
bag_clf_gridsearch.fit(X_train, Y_train)

[14:25:58] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_bylevel", "colsample_bynode", "colsample_bytree", "gamma", "grow_policy", "interaction_constraints", "max_bin", "max_cat_to_onehot", "max_delta_step", "max_depth", "max_leaves", "min_child_weight", "monotone_constraints", "num_parallel_tree", "predictor", "sampling_method", "subsample", "tree_method" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[14:25:58] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_bylevel", "colsample_bynode", "colsample_bytree", "gamma", "grow_policy", "interaction_constraints", "max_bin", "max_cat_to_onehot", "max_delta_step", "ma

c:\Users\Admin\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
15 fits failed out of a total of 320.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python37\lib\site-packages\xgboost\core.py", line 575, in inner_f
    return f(**kwargs)
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python37\lib\site-packages\xgboost\sklearn.py", line 972

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    callbacks=None, colsample_bylevel=1,
                                    colsample_bynode=1, colsample_bytree=1,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    gamma=0, gpu_id=-1, grow_policy='depthwise',
                                    importance_type=None,
                                    interaction_constraints='',
                                    learning_rate=0.300000012, max_bin=256,
                                    max_cat...step=0,
                                    max_depth=6, max_leaves=0,
                                    min_child_weight=1, missing=nan,
                                    monotone_constraints='()', n_estimators=100,
                                    n_jobs=0, num_parallel_tree=1,
     

In [9]:
bag_clf_gridsearch.best_params_

{'booster': 'gbtree', 'max_depth': 1, 'n_estimators': 150}

In [10]:
xgb_reg = xgboost.XGBRegressor(random_state=42,booster="gbtree",max_depth=1,n_estimators=150)

xgb_reg.fit(X_train, Y_train)
y_pred = xgb_reg.predict(X_test)
mean_absolute_error(Y_test, y_pred) 

259.0767999926291

In [11]:
xgb_reg.fit(X, Y)
y_pred = xgb_reg.predict(Z_test)

In [12]:
predictions_xgb_model=xgb_reg.predict(Z_test)

In [13]:
diccionario = {'id': Z_test['id'], 'Price_euros': np.array(predictions_xgb_model)}


entrega_xgb_model = pd.DataFrame(diccionario)


entrega_xgb_model

,id,Price_euros
0,824,1375.302002
1,101,715.786804
2,1157,1393.599976
3,940,969.145081
4,853,873.433167
...,...,...
386,292,963.729370
387,688,1199.361328
388,910,1521.721191
389,54,336.938354


In [ ]:
entrega_xgb_model.to_csv("entrega_xgb_model.csv",index=False)